In [5]:
pip install wikipedia azure-ai-inference azure-core requests


  Using cached azure_core-1.34.0-py3-none-any.whl.metadata (42 kB)
Using cached azure_core-1.34.0-py3-none-any.whl (207 kB)

   ---------------------------------------- 0/3 [isodate]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   ------------- -------------------------- 1/3 [azure-core]
   -------------------------- ------------- 2/3 [azure-ai-inference]
   -------------------------- ------------- 2/3 [azure-ai-inference]
   -------------------------- ------------- 2/3 [azure-ai-inference]


In [9]:
import wikipedia
import logging
from time import sleep
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import requests

# Suppress verbose Azure SDK logs
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Azure credentials
foundry_api_key = "BctGYR2wf2MXukgn2Ysx8DAHqp487Z5FByIYBNdcQlDx791DYbmGJQQJ99BDACfhMk5XJ3w3AAAAACOGjfH6"
gpt4o_api_key = "BctGYR2wf2MXukgn2Ysx8DAHqp487Z5FByIYBNdcQlDx791DYbmGJQQJ99BDACfhMk5XJ3w3AAAAACOGjfH6"

# Endpoints
foundry_endpoint = "https://hetar-m9lf056q-swedencentral.services.ai.azure.com/models"
gpt4o_endpoint = "https://hetar-m9lf056q-swedencentral.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"

# Initialize Foundry client
foundry_client = ChatCompletionsClient(
    endpoint=foundry_endpoint,
    credential=AzureKeyCredential(foundry_api_key),
    api_version="2024-05-01-preview"
)

# Wikipedia articles
article_titles = [
    "Artificial intelligence",
    "Climate change",
    "World War II",
    "Quantum computing",
    "Human brain"
]

def fetch_article(title):
    try:
        logging.info(f"Fetching: {title}")
        return wikipedia.page(title).content
    except Exception as e:
        logging.error(f"Failed to fetch {title}: {e}")
        return None

def summarize_foundry(model_name, text):
    try:
        response = foundry_client.complete(
            messages=[
                SystemMessage(content="You are a helpful assistant."),
                UserMessage(content=f"Summarize the following text:\n{text}")
            ],
            max_tokens=2048,
            model=model_name
        )
        return response.choices[0].message.content
    except Exception as e:
        logging.error(f"{model_name} failed: {e}")
        return None

def summarize_gpt4o(text):
    headers = {
        "Content-Type": "application/json",
        "api-key": gpt4o_api_key
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following text:\n{text}"}
        ],
        "temperature": 0.7
    }
    try:
        response = requests.post(gpt4o_endpoint, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        logging.error(f"GPT-4o failed: {e}")
        return None

# Main loop
for title in article_titles:
    text = fetch_article(title)
    if not text:
        continue
    text = text[:3000]  # Truncate for safety
    print(f"\n\n=== {title.upper()} ===")
    print("\n[MAI-DS-R1 Summary]:")
    print(summarize_foundry("MAI-DS-R1", text))
    print("\n[Phi-4-reasoning Summary]:")
    print(summarize_foundry("Phi-4-reasoning", text))
    print("\n[GPT-4o Summary]:")
    print(summarize_gpt4o(text))
    sleep(1)


INFO: Fetching: Artificial intelligence




=== ARTIFICIAL INTELLIGENCE ===

[MAI-DS-R1 Summary]:
<think>
Okay, I need to summarize this text about artificial intelligence. Let me start by reading through it carefully to understand the main points. The text defines AI, talks about its applications, subfields, historical development, and current issues like ethical concerns. 

First, the summary should begin with a definition of AI as computational systems performing tasks that require human intelligence. Then, mention the key areas like learning, reasoning, problem-solving, etc. It's important to note that AI is a part of computer science focused on creating machines that perceive their environment and act to achieve goals.

Next, the applications section lists things like search engines, recommendation systems, virtual assistants, autonomous vehicles, generative tools, and game-playing AI. I should group these examples concisely. Also, the text mentions that many AI applications aren't labeled as AI once they become common, w

INFO: Fetching: Climate change




=== CLIMATE CHANGE ===

[MAI-DS-R1 Summary]:
<think>
Okay, I need to summarize the given text about climate change. Let me start by reading through the text carefully to understand the main points.

The first paragraph discusses present-day climate change, including global warming and its causes. It mentions that human activities, especially burning fossil fuels since the Industrial Revolution, are the main drivers. The paragraph also notes the increase in greenhouse gases like CO2, which has risen by 50% since pre-industrial times.

The second paragraph talks about the environmental impacts: expanding deserts, more heat waves and wildfires, Arctic changes like melting permafrost and sea ice decline, intense storms, droughts, and species relocation or extinction. It also mentions long-term effects like ocean heating, acidification, and sea level rise even if warming is minimized.

The third paragraph addresses the threats to people, such as flooding, heat, food/water scarcity, diseas

INFO: Fetching: World War II




=== WORLD WAR II ===

[MAI-DS-R1 Summary]:
<think>
Okay, I need to summarize the provided text about World War I. Let me read through it carefully first. The text starts by stating the duration of the war, the coalitions involved (Allies vs. Central Powers), and the main regions where fighting occurred. It mentions trench warfare, the use of new weapons like tanks and chemical weapons, and the high casualties. There's also a part about the Spanish flu pandemic being exacerbated by the movement of troops.

Next, the causes are discussed: the rise of Germany, decline of the Ottoman Empire, economic competition, and the assassination of Archduke Franz Ferdinand. The text details how the war expanded with various countries joining in, key strategies like Germany's Schlieffen Plan, and the stalemate on the Western Front. Important battles like Verdun and the Somme are mentioned, along with the entry of the US in 1917, Russia's exit post-Bolshevik Revolution, and the eventual collapse of t

INFO: Fetching: Quantum computing




=== QUANTUM COMPUTING ===

[MAI-DS-R1 Summary]:
<think>
Okay, I need to summarize the given text about quantum computers. Let me read through it carefully first. The text explains what a quantum computer is, how it uses qubits instead of classical bits, the challenges in building practical quantum computers, and some historical context.

First, I should identify the main points. The key elements are:

1. Quantum computers use quantum mechanical phenomena.
2. Qubits can be in superposition, unlike classical bits.
3. Quantum computers have potential advantages in speed for certain problems.
4. Current challenges include decoherence, error rates, and scalability.
5. Quantum supremacy is a theoretical advantage but not yet practical.
6. History mentions the development of quantum theory and its intersection with computer science.

I need to condense these points without getting too technical. The summary should start with a definition of a quantum computer, mention qubits and superpositi

INFO: Fetching: Human brain




=== HUMAN BRAIN ===

[MAI-DS-R1 Summary]:
<think>
Okay, let's tackle this summary. The user wants a concise summary of the given text about the human brain. First, I need to identify the main points. The text starts by introducing the brain as the central organ of the nervous system, working with the spinal cord. Then it breaks down the brain's structure: cerebrum, brainstem, cerebellum. The cerebrum's hemispheres, lobes, and their functions are detailed. The brainstem components are midbrain, pons, medulla. The cerebellum connects via peduncles. Also mentioned are the ventricular system, subcortical structures like thalamus, hypothalamus, limbic system, and cell types (neurons and glial cells). Protection mechanisms like skull, CSF, and blood-brain barrier are noted.

I should avoid too much detail. For example, the specific layers of the neocortex and allocortex might be too granular. Focus on major parts and their functions. Emphasize the cerebrum's lobes and their roles, the brai